In [1]:
import numpy as np

from pathlib import Path
from warnings import simplefilter

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from sklearn.metrics import mean_squared_error, r2_score

#data_dir = Path("../input/ts-course-data")
Flav_sales_train = pd.read_csv(
    "/Users/sunilsavanur/Documents/Kaggles/Flavorita_store_prediction/train.csv", 
    usecols=['id','store_nbr', 'family', 'date', 'sales', 'onpromotion'],
    parse_dates=["date"])


#now read test.csv for forecast
df_test = pd.read_csv(
    '/Users/sunilsavanur/Documents/Kaggles/Flavorita_store_prediction/test.csv',
    usecols=['id','store_nbr', 'family', 'date', 'onpromotion'],
    parse_dates=['date']
)

Flav_sales_train['date'] = Flav_sales_train.date.dt.to_period('D')
Flav_sales_train = Flav_sales_train.set_index(['date','store_nbr','family']).sort_index()
total_sales = (
    Flav_sales_train.groupby(['date']).mean().squeeze()
)
df_test['date'] = df_test.date.dt.to_period('D')
df_test = df_test.set_index(['date','store_nbr','family']).sort_index()


total_sales = total_sales.drop(['id','onpromotion'],axis=1)
total_sales = total_sales.dropna(subset = ['sales'])

column_headers = list(total_sales.columns.values)
print("The Column Header :", column_headers)

#display(total_sales)
print("----- earth quake -----")
#display(total_sales.loc['2016-04-14'])
#display(total_sales.loc['2016-04-15'])
display(total_sales.loc['2016-04-16'])
#display(total_sales.loc['2016-04-17'])
#display(total_sales.loc['2016-04-18'])
#display(total_sales.loc['2016-04-19'])
#display(total_sales.loc['2016-04-20'])
#display(total_sales.loc['2016-04-21'])
#display(total_sales.loc['2016-04-22'])
#display(total_sales.loc['2016-04-23'])
#display(total_sales.loc['2016-04-24'])
#display(total_sales.loc['2016-04-25'])
#display(total_sales.loc['2016-04-26'])
display(total_sales.loc['2016-04-27'])

drop_list = ['2016-04-16','2016-04-17','2016-04-18','2016-04-19','2016-04-20','2016-04-21',
            '2016-04-22','2016-04-23','2016-04-24','2016-04-25','2016-04-26']
drop_sales =  total_sales.loc[drop_list]
display(drop_sales)
total_sales = total_sales.drop(drop_sales.index)

Flav_sales_train = Flav_sales_train.drop(drop_sales.index)
print("----- after dropping earth quake date index -----")
display(total_sales)

moving_average = total_sales.rolling(
    window=365,       # 365-day window
    center=True,      # puts the average at the center of the window
    min_periods=183,  # choose about half the window size
).mean()              # compute the mean (could also do median, std, min, max, ...)

fourier = CalendarFourier(freq="A", order=4)  # 4 sin/cos pairs for 'M'onthly / "A"nnual seasonality
dp = DeterministicProcess(
    index=total_sales.index,
    constant=True,               # dummy feature for bias (y-intercept)
    order=1,                     # trend (order 1 means linear)
    seasonal=True,               # weekly seasonality (indicators)
    additional_terms=[fourier],  # annual seasonality (fourier)
    drop=True,                   # drop terms to avoid collinearity
)

y_train = Flav_sales_train.unstack(['store_nbr', 'family'])
X_train = dp.in_sample()  # create features for dates in total_sales.index
X_train['NewYear'] = (X_train.index.dayofyear == 1)
#display(X.head)

model = LinearRegression().fit(X_train, y_train)

# Create features for test set
X_test = dp.out_of_sample(steps=16)
X_test.index.name = 'date'
X_test['NewYear'] = (X_test.index.dayofyear == 1)

#model = LinearRegression().fit(X, y)
#y_pred = pd.DataFrame(model.predict(X), index=X.index, columns=y.columns)

# -----------------------    --------------------
# Make predictions
y_fit = pd.DataFrame(
    model.predict(X_train),
    index=y_train.index,
    columns=y_train.columns,
)
y_pred = pd.DataFrame(
    model.predict(X_test),
    index=X_test.index,
    columns=y_train.columns,
)
y_resid = y_train - y_fit

print("----- y_fit -----")
#display(y_fit)
print("----- y_pred -----")
display(y_pred)

# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_train, y_fit,squared=True))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_train, y_fit))

#display(df_test)
y_submit = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=y_train.columns)
y_submit = y_submit.stack(['store_nbr', 'family'])
y_submit = y_submit.join(df_test.id,how = 'left', lsuffix='_left', rsuffix='_right')
y_submit = y_submit.rename(columns={'id_right': 'id'})

y_submit_final = y_submit.set_index(['id']).sort_index()
y_submit_final = y_submit_final.drop(['id_left','onpromotion'],axis=1)
#display(y_submit)
display(y_submit_final)
y_submit_final.to_csv('/Users/sunilsavanur/Documents/Kaggles/Flavorita_store_prediction/submission.csv', index=True)

The Column Header : ['sales']
----- earth quake -----


sales    483.794328
Name: 2016-04-16, dtype: float64

sales    376.143714
Name: 2016-04-27, dtype: float64

,sales
date,
2016-04-16,483.794328
2016-04-17,713.711414
2016-04-18,755.286535
2016-04-19,646.514707
2016-04-20,596.198816
2016-04-21,561.773165
2016-04-22,480.953560
2016-04-23,573.593229
2016-04-24,583.260343


----- after dropping earth quake date index -----


,sales
date,
2013-01-01,1.409438
2013-01-02,278.390807
2013-01-03,202.840197
2013-01-04,198.911154
2013-01-05,267.873244
...,...
2017-08-11,463.733851
2017-08-12,444.798280
2017-08-13,485.768618


----- y_fit -----
----- y_pred -----


id                                            \
store_nbr             1                                              
family        AUTOMOTIVE     BABY CARE        BEAUTY     BEVERAGES   
date                                                                 
2017-08-16  2.999264e+06  2.999265e+06  2.999266e+06  2.999267e+06   
2017-08-17  3.001115e+06  3.001116e+06  3.001117e+06  3.001118e+06   
2017-08-18  3.002871e+06  3.002872e+06  3.002873e+06  3.002874e+06   
2017-08-19  3.004627e+06  3.004628e+06  3.004629e+06  3.004630e+06   
2017-08-20  3.006379e+06  3.006380e+06  3.006381e+06  3.006382e+06   
2017-08-21  3.008137e+06  3.008138e+06  3.008139e+06  3.008140e+06   
2017-08-22  3.009891e+06  3.009892e+06  3.009893e+06  3.009894e+06   
2017-08-23  3.011653e+06  3.011654e+06  3.011655e+06  3.011656e+06   
2017-08-24  3.013499e+06  3.013500e+06  3.013501e+06  3.013502e+06   
2017-08-25  3.015252e+06  3.015253e+06  3.015254e+06  3.015255e+06   
2017-08-26  3.017004e+06  3.017005e+06  3.017006e+06  3.017007e+06   
2017-08-27  3.018752e+06  3.018753e+06  3.018754e+06  3.018755e+06   
2017-08-28  3.020508e+06  3.020509e+06  3.020510e+06  3.020511e+06   
2017-08-29  3.022259e+06  3.022260e+06  3.022261e+06  3.022262e+06   
2017-08-30  3.024019e+06  3.024020e+06  3.024021e+06  3.024022e+06   
2017-08-31  3.025864e+06  3.025865e+06  3.025866e+06  3.025867e+06   

                                                                    \
store_nbr                                                            
family             BOOKS  BREAD/BAKERY   CELEBRATION      CLEANING   
date                                                                 
2017-08-16  2.999268e+06  2.999269e+06  2.999270e+06  2.999271e+06   
2017-08-17  3.001119e+06  3.001120e+06  3.001121e+06  3.001122e+06   
2017-08-18  3.002875e+06  3.002876e+06  3.002877e+06  3.002878e+06   
2017-08-19  3.004631e+06  3.004632e+06  3.004633e+06  3.004634e+06   
2017-08-20  3.006383e+06  3.006384e+06  3.006385e+06  3.006386e+06   
2017-08-21  3.008141e+06  3.008142e+06  3.008143e+06  3.008144e+06   
2017-08-22  3.009895e+06  3.009896e+06  3.009897e+06  3.009898e+06   
2017-08-23  3.011657e+06  3.011658e+06  3.011659e+06  3.011660e+06   
2017-08-24  3.013503e+06  3.013504e+06  3.013505e+06  3.013506e+06   
2017-08-25  3.015256e+06  3.015257e+06  3.015258e+06  3.015259e+06   
2017-08-26  3.017008e+06  3.017009e+06  3.017010e+06  3.017011e+06   
2017-08-27  3.018756e+06  3.018757e+06  3.018758e+06  3.018759e+06   
2017-08-28  3.020512e+06  3.020513e+06  3.020514e+06  3.020515e+06   
2017-08-29  3.022263e+06  3.022264e+06  3.022265e+06  3.022266e+06   
2017-08-30  3.024023e+06  3.024024e+06  3.024025e+06  3.024026e+06   
2017-08-31  3.025868e+06  3.025869e+06  3.025870e+06  3.025871e+06   

                                        ... onpromotion            \
store_nbr                               ...          54             
family             DAIRY          DELI  ...   MAGAZINES     MEATS   
date                                    ...                         
2017-08-16  2.999272e+06  2.999273e+06  ...    0.038664  3.765206   
2017-08-17  3.001123e+06  3.001124e+06  ...    0.025931  4.251569   
2017-08-18  3.002879e+06  3.002880e+06  ...    0.029752  2.922333   
2017-08-19  3.004635e+06  3.004636e+06  ...    0.033505  1.629730   
2017-08-20  3.006387e+06  3.006388e+06  ...    0.058046  1.630955   
2017-08-21  3.008145e+06  3.008146e+06  ...    0.053371  1.643999   
2017-08-22  3.009899e+06  3.009900e+06  ...    0.040205  2.298183   
2017-08-23  3.011661e+06  3.011662e+06  ...    0.035434  3.770751   
2017-08-24  3.013507e+06  3.013508e+06  ...    0.022278  4.250162   
2017-08-25  3.015260e+06  3.015261e+06  ...    0.025703  2.914103   
2017-08-26  3.017012e+06  3.017013e+06  ...    0.029092  1.614844   
2017-08-27  3.018760e+06  3.018761e+06  ...    0.053303  1.609614   
2017-08-28  3.020516e+06  3.020517e+06  ...    0.048334  1.616441   
2017-08-29  3.022267e+06  3.022268e+06  ...    

Mean squared error: 9810420.62
Coefficient of determination: 0.54


,sales
id,
3000888,4.905186
3000889,0.000000
3000890,3.476591
3000891,2396.281856
3000892,0.301408
...,...
3029395,409.048503
3029396,111.806594
3029397,1892.647457
